In [1]:

from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import mean_squared_error

In [2]:
# run xgboost 
def tree_train(x,y):
    parameters = {"max_depth":[3,4,5,6,7,8,9], "min_child_weight":[1,2,3]}
    model = XGBRegressor()
    gs = GridSearchCV(model,param_grid=parameters,cv=10)
    gs.fit(x,y)
    return gs





In [18]:
years = []
cur_year = '2018/19'
def is_next_year(first,second):
    print(first)
    yy1 = first.split('/')[1]
    yy2 = second.split('/')[0][2:]
    return yy1 == yy2

In [58]:
def read_data(file_name='../fpl_data/data/player_history.csv'):
    global years
    clean_df = pd.read_csv(file_name,encoding = "ISO-8859-1")
    years = list(clean_df['season'].unique())
    year_id = clean_df[['id','total_points','season']]
    year_id['total_points_next'] =year_id['total_points']
    year_id['season_next'] = year_id['season']
    year_id.drop(['total_points','season'],axis=1,inplace=True)
    clean_df = pd.merge(clean_df, year_id,on='id') 
    
    def is_keep(years):

        first =  years['season']
        second = years['season_next']
        return first-second-1
    clean_df['keep'] = clean_df.apply(is_keep,axis=1)
    #print(clean_df['keep'])
    clean_df = clean_df.query('keep==0')
    clean_df.drop('keep',axis=1,inplace=True)
    
    
    return clean_df

In [59]:
df_summary = read_data()

/home/shibin/softwares/anaconda2/envs/aiproject/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/shibin/softwares/anaconda2/envs/aiproject/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/shibin/softwares/anaconda2/envs/aiproject/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://

In [60]:
df_summary.shape

(1227, 31)

# Create train and test data. Data sample is less

In [113]:

def split_t_t(df):
    cur_season = 11
    train_df = df.query('season<11')
    test_df = df.query('season==11')
    return train_df,test_df
def clean_data(x,cols):
    unwanted_cols = ['first_name','second_name','total_points_next','season','season_name','id','season_next']
    for c in list(x):
        if c not in cols:
            unwanted_cols.append(c)
    return x.drop(unwanted_cols,axis=1)

In [114]:
train_df,test_df = split_t_t(df_summary)
train_y =train_df['total_points_next']
test_y = test_df['total_points_next']
test_x = clean_data(test_df,list(test_df))
train_x = clean_data(train_df,list(train_df))

#print(list(train_x))

In [115]:
list(train_x)

['yellow_cards',
 'goals_conceded',
 'saves',
 'influence',
 'goals_scored',
 'own_goals',
 'creativity',
 'end_cost',
 'ict_index',
 'total_points',
 'penalties_missed',
 'start_cost',
 'element_code',
 'bonus',
 'clean_sheets',
 'assists',
 'ea_index',
 'penalties_saved',
 'minutes',
 'bps',
 'element_type',
 'threat',
 'team',
 'red_cards']

# Run training

In [116]:
clf = tree_train(train_x,train_y)

/home/shibin/softwares/anaconda2/envs/aiproject/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [117]:
prediction = clf.predict(test_x)

In [118]:
error = mean_squared_error(prediction,test_y)

In [119]:
print(error)

997.6710287478684


In [120]:
def write_result(test_x,test_y,predicted,file='next_year.csv'):
    first_name =test_x['first_name']
    second_name = test_x['second_name']
    df = pd.DataFrame()
    df['first_name'] = first_name
    df['second_name'] = second_name
    df['actual'] = test_y
    df['predicted'] = predicted
    df['element_type'] = test_x['element_type']
    df['diff'] = (df['actual']-df['predicted']).abs()
    df.to_csv(file,index=False)
    return df
    

In [121]:
final_combined = write_result(test_df,test_y,prediction)

# Get players

In [122]:
def select_players(df):
    df = df.sort_values(by=['predicted'],ascending=False)
    result = [[] for x in range(4)]
    required = [2,5,5,3]
    for index,row in df.iterrows():
        f_name = row['first_name']
        s_name = row['second_name']
        pred = row['predicted']
        actual = row['actual']
        e_index = int(row['element_type'])-1
        if(len(result[e_index])<required[e_index]):
            item = {}
            item['first_name'] =f_name
            item['second_name'] =  s_name
            item['prediction'] = pred
            item['actual'] = actual
            #itemp['element_type'] = row['element_type']
            
            result[e_index].append(item)     
    return result

def actual_players(df):
    df = df.sort_values(by=['actual'],ascending=False)
    result = [[] for x in range(4)]
    required = [2,5,5,3]
    for index,row in df.iterrows():
        f_name = row['first_name']
        s_name = row['second_name']
        pred = row['predicted']
        actual = row['actual']
        e_index = int(row['element_type'])-1
        if(len(result[e_index])<required[e_index]):
            item = {}
            item['first_name'] =f_name
            item['second_name'] =  s_name
            item['prediction'] = pred
            item['actual'] = actual
            #itemp['element_type'] = row['element_type']
            
            result[e_index].append(item)     
    return result
def print_result(result):
    types = ['Goali','Defender','Mid','Attacker']
    for i in range(len(result)):
        print('==================SELECTED {}==================='.format(types[i]))
        for player in result[i]:
            print('{},{}\t\t\t,{},\t{}\n'.format(player['first_name'],player['second_name'],player['prediction'],player['actual']))
            


In [123]:
s_list = select_players(final_combined)
a_list =  actual_players(final_combined)

# Selected by prediction

In [124]:
print_result(s_list)

==================SELECTED Goali===================
Heurelho,Gomes			,134.01930236816406,	157

Petr,Cech			,130.5564422607422,	159

==================SELECTED Defender===================
Leighton,Baines			,133.3016357421875,	67

Toby,Alderweireld			,112.4158706665039,	166

Gary,Cahill			,110.19263458251953,	53

NicolÃ¡s,Otamendi			,109.75376892089844,	104

Laurent,Koscielny			,108.62327575683594,	153

==================SELECTED Mid===================
Riyad,Mahrez			,194.32177734375,	240

Mesut,Ãzil			,183.43150329589844,	200

Kevin,De Bruyne			,175.80723571777344,	131

Alexis,SÃ¡nchez			,168.58709716796875,	167

Eden,Hazard			,163.53981018066406,	104

==================SELECTED Attacker===================
Jamie,Vardy			,192.6436004638672,	211

Sergio,AgÃ¼ero			,171.1587677001953,	184

Roberto,Firmino			,163.55015563964844,	155



# Actual  Performance

In [125]:
print_result(a_list)

==================SELECTED Goali===================
Petr,Cech			,130.5564422607422,	159

Heurelho,Gomes			,134.01930236816406,	157

==================SELECTED Defender===================
HÃ©ctor,BellerÃ­n			,106.08972930908203,	172

Toby,Alderweireld			,112.4158706665039,	166

Laurent,Koscielny			,108.62327575683594,	153

Nacho,Monreal			,106.56748962402344,	152

Christian,Fuchs			,106.10931396484375,	150

==================SELECTED Mid===================
Riyad,Mahrez			,194.32177734375,	240

Mesut,Ãzil			,183.43150329589844,	200

Christian,Eriksen			,137.55416870117188,	178

Georginio,Wijnaldum			,122.26779174804688,	170

Alexis,SÃ¡nchez			,168.58709716796875,	167

==================SELECTED Attacker===================
Jamie,Vardy			,192.6436004638672,	211

Harry,Kane			,155.7584991455078,	211

Romelu,Lukaku			,146.43580627441406,	185



In [127]:
print_result(s_list)

==================SELECTED Goali===================
Heurelho,Gomes			,134.01930236816406,	157

Petr,Cech			,130.5564422607422,	159

==================SELECTED Defender===================
Leighton,Baines			,133.3016357421875,	67

Toby,Alderweireld			,112.4158706665039,	166

Gary,Cahill			,110.19263458251953,	53

NicolÃ¡s,Otamendi			,109.75376892089844,	104

Laurent,Koscielny			,108.62327575683594,	153

==================SELECTED Mid===================
Riyad,Mahrez			,194.32177734375,	240

Mesut,Ãzil			,183.43150329589844,	200

Kevin,De Bruyne			,175.80723571777344,	131

Alexis,SÃ¡nchez			,168.58709716796875,	167

Eden,Hazard			,163.53981018066406,	104

==================SELECTED Attacker===================
Jamie,Vardy			,192.6436004638672,	211

Sergio,AgÃ¼ero			,171.1587677001953,	184

Roberto,Firmino			,163.55015563964844,	155

